In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
# load dataset
data = pd.read_csv('../input/spotify/spotify final datasets.csv')
data.head()

### Data structure

# EDA

In [ ]:
type(data)

In [ ]:
# check how many rows and columns
data.shape

In [ ]:
# data types
data.dtypes

In [ ]:
data.info()

In [ ]:
data[data.duplicated()]

In [ ]:
# Remove duplicated rows
data1 = data.drop_duplicates()
data1.duplicated().sum()

In [ ]:
# Remove the unnecessary columans
data_clean = data1.drop('uri',axis =1)
data_clean.head()

In [ ]:
data2 = data_clean.drop('track_number', axis =1)
data2.head()

In [ ]:
# checking for null value present or not
data2.isnull().sum()

In [ ]:
df = data2.fillna(data2.mode().iloc[0])

In [ ]:
df.isnull().sum()

In [ ]:
# to check corr
corr = df.corr()
corr

In [ ]:
# create heat map using corr
sns.heatmap(corr, xticklabels = corr.columns, yticklabels=corr.columns, annot  = True)

In [ ]:
df.describe()

In [ ]:
df.nunique()

###  Visualization 

In [ ]:
sns.pairplot(df)

In [ ]:
df['popularity'].hist()

In [ ]:
df.boxplot(column = ['popularity'])

# Model Building

In [ ]:
df.head()

In [ ]:
popu = pd.DataFrame(df.groupby('name')['popularity'].mean())
popu.head()

In [ ]:
matrics= df.pivot_table(index = 'album', columns = 'name', values ='popularity')
matrics

In [ ]:
matrix_2 = matrics.fillna(0)
matrix_2

In [ ]:
popu.sort_values('popularity', ascending = False).head(10)

In [ ]:
a = matrix_2["That's The Power - Live"]

In [ ]:
similar_to_a = matrix_2.corrwith(a)
similar_to_a

In [ ]:
corr_a = pd.DataFrame(similar_to_a , columns=['corr'])
corr_a.dropna(inplace=True)
corr_a.head()

In [ ]:
corr_a_2 = corr_a.join(popu['popularity'])
corr_a_2

In [ ]:
corr_a_2[corr_a_2['popularity']>20].sort_values(by='corr', ascending = False)

In [ ]:
pickle.dump(df.to_dict(),open('spotify.pkl','wb'))

# Model Evoluation

In [ ]:
# create new column 'rating'
df['popularity'] = df['rating']= data.popularity.map(lambda x: 1 if x > 5 else 0)
df.head()

In [ ]:
# drop unnecessary columns
df1 = df.drop(['popularity'], axis = 1)
df1.head()

In [ ]:
df1['rating'].value_counts()

In [ ]:
d1 = df1.drop('id',axis =1)
d1

In [ ]:
x = d1.drop('rating',axis = 1)
y = d1['rating']

In [ ]:
# create dummies for following columns
x1 = pd.get_dummies(x, columns =['acousticness','danceability','energy','instrumentalness','liveness','loudness','speechiness','tempo','valence','album','name'])
x1.head()

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x1,y, test_size = 0.25, random_state =0)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clas = LogisticRegression()

In [ ]:
clas.fit(x_train,y_train)

In [ ]:
y_pred = clas.predict_proba(x_test)
y_pred

In [ ]:
y_pred = clas.predict(x_test)
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix

#### Confusion Matrix
Confusion matrix is one of the most powerful and commonly used evaluation technique as it allows us to compute a whole lot of other metrics that allow us to evaluate the performance of a classification model.

In [ ]:
# Creating a simple confusion matrix
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
# We will now represent the Confusion Matrix in a heat map format by using the seaborn package.
sns.heatmap(cm, annot = True, fmt = '.2f')
plt.ylabel('True label')
plt.xlabel('predicted label')

###### Various types of metrics can be calculated using the confusion matrix and in order to do so, we create a classification report.

In [ ]:
from sklearn import metrics

In [ ]:
print(metrics.classification_report(y_test, y_pred, digits = 2))

#### Accuracy Score
Accuracy Score can be calculated using metrics.

In [ ]:
metrics.accuracy_score(y_test,y_pred)

#### The accuracy score for the logistic regression model comes out to be 0.80

### AUC and ROC

In [ ]:
# AUC
predict_proba = pd.DataFrame(clas.predict_proba(x_test))
predict_proba.head()

In [ ]:
# We will now make a dataset containing actual values of the Survived variable (dependent variable), predicted values and probabilities of survival. To do this, we first start with converting predicted values of survival to a data frame for merging datasets.
pred_log = pd.DataFrame(y_pred)

In [ ]:
# We now reset the index for y_test
y1_test = y_test.reset_index()

In [ ]:
# We then concatenate datasets using pd.concat
predictions = pd.concat([y1_test,pred_log,predict_proba],axis = 1)

In [ ]:
# Finally, the columns of the dataset are renamed and we get the final table that allows us to calculate the AUC score and create ROC Curve.
predictions.columns = ["index","actual","prediction","survived_0","survived_1"]

In [ ]:
predictions.head()

In [ ]:
# We use the above table to compute the AUC score i.e. the Area Under the Curve.
auc_score = metrics.roc_auc_score(predictions.actual,predictions.survived_1)
round(float(auc_score),2)

In [ ]:
# We now calculate the Fals Positivity Rate, True Positivity Rate and Threshold and use them to plot the ROC curve(Receiver Operating Characteristic).
fpr,tpr,threshold = metrics.roc_curve(y_test,predictions.survived_1,drop_intermediate =False)
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label='ROC curve (area = %0.2f)' % auc_score)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

As we can notice, the minimum difference between the False Positive and True Positive is when our sensitivity value is at 0.8. Now we will calculate the new cut off value based on this value of sensitivity and see how the accuracy of our model increases.

In [ ]:
cutoff_prob = threshold[(np.abs(tpr - 0.8 )).argmin()]
round( float( cutoff_prob), 2)

###### The ideal cutoff for having the maximum sensitivity (True Positive Rate) and 1-specificity (False Positive Rate) comes out to be 0.28

In [ ]:
# We will now predict the survival rate by using the new cut off value.
predictions['new_labels'] = predictions['survived_1'].map( lambda x: 1 if x >= 0.28 else 0 )
predictions.head()

In [ ]:
# We create new Confusion Matrix with actual and new values.
cm1 = metrics.confusion_matrix( predictions.actual,
                          predictions.new_labels, [1,0] )
sns.heatmap(cm1, annot=True,  fmt='.2f', xticklabels = ["No", "Yes"] , yticklabels = ["No", "Yes"],)
plt.ylabel('True label',fontsize=12)
plt.xlabel('Predicted label',fontsize=12)

In [ ]:
# We also calculate the new Accuracy Score.
metrics.accuracy_score(predictions.actual,predictions.new_labels, [1,0] )

### We notice a slight increase in our accuracy from roughly 0.80 to 0.86¶

### Deployment